In [75]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [76]:
# Load and preprocess the data
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
column_names = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']
df = pd.read_csv(url, names=column_names)
print(df.head())
print(df.info())
df[['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']] = df[['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']].replace(0, np.NaN)
df.fillna(df.mean(), inplace=True)
X = df.drop('Outcome', axis=1)
y = df['Outcome']
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   
2            8      183             64              0        0  23.3   
3            1       89             66             23       94  28.1   
4            0      137             40             35      168  43.1   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0  
2                     0.672   32        1  
3                     0.167   21        0  
4                     2.288   33        1  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768

In [77]:
# Train the model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print(classification_report(y_test, y_pred))

Accuracy: 0.75
              precision    recall  f1-score   support

           0       0.81      0.81      0.81        99
           1       0.65      0.65      0.65        55

    accuracy                           0.75       154
   macro avg       0.73      0.73      0.73       154
weighted avg       0.75      0.75      0.75       154



In [78]:
# Step 3: Define Functions for Diet Recommendation
def categorize_patient(bmi, glucose, age):
    if glucose >= 200:
        risk = "High"
    elif glucose >= 140:
        risk = "Moderate"
    else:
        risk = "Low"

    if bmi >= 30:
        weight_category = "Obese"
    elif bmi >= 25:
        weight_category = "Overweight"
    else:
        weight_category = "Normal"

    if age >= 60:
        age_group = "Senior"
    elif age >= 45:
        age_group = "Middle-aged"
    else:
        age_group = "Young"

    return risk, weight_category, age_group

In [79]:

def recommend_diet_enhanced(risk, weight_category, age_group):
    if risk == "High":
        diet = "Strict low-carb diet, avoid refined sugars, include high-fiber foods, leafy greens, and lean proteins."
        if weight_category == "Obese":
            diet += " Focus on portion control and calorie deficit. Include regular physical activity."
        if age_group == "Senior":
            diet += " Consider foods rich in calcium and vitamin D for bone health."
    elif risk == "Moderate":
        diet = "Moderate low-carb diet, whole grains, and vegetables. Monitor portion sizes."
        if weight_category == "Overweight":
            diet += " Include more fruits, vegetables, and lean proteins. Aim for a balanced calorie intake."
        if age_group == "Middle-aged":
            diet += " Include heart-healthy fats like omega-3s."
    else:
        diet = "Balanced diet, include a variety of whole grains, fruits, vegetables, lean proteins, and healthy fats."
        if weight_category == "Normal" and age_group == "Young":
            diet += " Maintain a balanced lifestyle with regular physical activity."
        if weight_category == "Normal" and age_group == "Senior":
            diet += " Focus on maintaining bone health with calcium and vitamin D."

    return diet


In [80]:
def detailed_nutritional_guidelines(diet):
    if "low-carb" in diet:
        return "Focus on consuming vegetables like broccoli, spinach, kale, and cauliflower. Lean proteins like chicken, turkey, and tofu. Healthy fats like avocados, nuts, and olive oil."
    elif "balanced" in diet:
        return "Ensure a mix of macronutrients in each meal. Include whole grains like quinoa, brown rice, or oats. Incorporate fruits like berries, apples, and oranges. Lean proteins like fish, eggs, and legumes."
    else:
        return "General diet: Ensure hydration, eat three balanced meals a day, and include snacks like nuts or fruits."

In [81]:
# Step 4: Predict Diabetes and Recommend Diet
diet_recommendation=""
def predict_diabetes_and_recommend_diet(new_patient_data):
    # Normalize the new patient data using the same scaler
    new_patient = scaler.transform(new_patient_data)

    # Predict diabetes
    diabetes_prediction = model.predict(new_patient)[0]
    diabetes_probability = model.predict_proba(new_patient)[0]

    # Extract features for diet recommendation
    bmi = new_patient_data[0][5]  # Assuming BMI is the 6th feature
    glucose = new_patient_data[0][1]  # Assuming Glucose is the 2nd feature
    age = new_patient_data[0][7]  # Assuming Age is the 8th feature

    risk, weight_category, age_group = categorize_patient(bmi, glucose, age)
    global diet_recommendation
    diet_recommendation  = recommend_diet_enhanced(risk, weight_category, age_group)
    diet_guideline = detailed_nutritional_guidelines(diet_recommendation)

    # Print the prediction and recommendation
    if diabetes_prediction == 1:
        print("The patient is predicted to have diabetes.")
    else:
        print("The patient is predicted to not have diabetes.")

    print(f"Probability of having diabetes: {diabetes_probability[1]:.2f}")
    print(f"Probability of not having diabetes: {diabetes_probability[0]:.2f}")
    print(f"Diet Recommendation: {diet_recommendation}")
    print(f"Nutritional Guidelines: {diet_guideline}")

In [82]:
# Step 4: Define Dish Suitability
dish_suitability = {
    "salad": "good",
    "grilled chicken": "good",
    "white rice": "bad",
    "pizza": "bad",
    "quinoa": "good",
    "fruit salad": "good",
    "ice cream": "bad",
    "whole grain bread": "good",
    "pasta": "bad",
    "fish": "good",

}

def check_dish_suitability(dish_name):
    dish_name = dish_name.lower()
    suitability = dish_suitability.get(dish_name, "unknown")
    if suitability == "good":
        return f"The dish '{dish_name}' is generally good for a diabetic patient."
    elif suitability == "bad":
        return f"The dish '{dish_name}' is generally bad for a diabetic patient."
    else:
        return f"The suitability of the dish '{dish_name}' for a diabetic patient is unknown. Please consult a dietitian."

In [83]:
# Step 6: Test with New Patient Data
# Example of a new patient (input data in the same order as in the training data)
new_patient_data = np.array([[2, 1500, 850, 200, 80, 25.0, 0.5, 350]])

# Specify a dish name to check its suitability
dish_name = "burger"

In [84]:
predict_diabetes_and_recommend_diet(new_patient_data)

The patient is predicted to have diabetes.
Probability of having diabetes: 0.57
Probability of not having diabetes: 0.43
Diet Recommendation: Strict low-carb diet, avoid refined sugars, include high-fiber foods, leafy greens, and lean proteins. Consider foods rich in calcium and vitamin D for bone health.
Nutritional Guidelines: Focus on consuming vegetables like broccoli, spinach, kale, and cauliflower. Lean proteins like chicken, turkey, and tofu. Healthy fats like avocados, nuts, and olive oil.


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [85]:
diet_recommendation

'Strict low-carb diet, avoid refined sugars, include high-fiber foods, leafy greens, and lean proteins. Consider foods rich in calcium and vitamin D for bone health.'

In [86]:
check_dish_suitability(dish_name)

"The suitability of the dish 'burger' for a diabetic patient is unknown. Please consult a dietitian."

In [87]:
# Import necessary libraries
!pip install gtts
from gtts import gTTS
from IPython.display import Audio, display


# Language for the speech
language = 'en'

# Generate the speech
speech = gTTS(text=diet_recommendation, lang=language, slow=False)

# Save the speech to an audio file
speech.save("output.mp3")

# Load and display the audio in the notebook
display(Audio("output.mp3", autoplay=True))
